In [1]:
%load_ext autoreload
%autoreload 2

## KEGGapFiller Example

In [1]:
from refinegems.classes.gapfill import KEGGapFiller
from refinegems.utility.io import load_model

modelpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435.xml'
model = load_model(modelpath,'libsbml')
cmodel = load_model(modelpath,'cobra')

print(len(model.getPlugin(0).getListOfGeneProducts()))

gfk = KEGGapFiller('sha')

gfk_missing_genes = gfk.get_missing_genes(model)

gfk_missing_reac = gfk.get_missing_reacs(cmodel,gfk_missing_genes)

model = gfk.fill_model(model,gfk_missing_genes,gfk_missing_reac)

print(len(model.getPlugin(0).getListOfGeneProducts()))

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


787
Running in debugging mode.


100%|██████████| 20/20 [00:24<00:00,  1.23s/it]


fill_model: Running in debugging mode


Adding genes to model: 100%|██████████| 2/2 [00:00<00:00, 3636.15it/s]

789


In [2]:
gfk._statistics 

{'genes': {'missing (before)': 20, 'added': 2, 'missing (after)': 18},
 'reactions': {'added (total)': 7, 'failed to build': 3},
 'metabolites': {}}

## GeneGapFiller Example

In [1]:
from refinegems.classes.gapfill import GeneGapFiller
from refinegems.utility.io import load_model

modelpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435.xml'
model = load_model(modelpath,'libsbml')
cmodel = load_model(modelpath,'cobra')

gffpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_RefSeq.gff'

tfasta = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_proteins_genome.fasta'
spdb = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/swissprot.dmnd'
spmap = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/uniprot_table.tsv'
kwargs = {'outdir':'/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435',
          'sens':'more-sensitive',
          'cov':90.0,
          't':4,
          'pid':90.0}

gf2 = GeneGapFiller()
gf2_missing_genes = gf2.get_missing_genes(gffpath,model)
# ncbiprotein | locus_tag | ec-code
mapped_res = gf2.get_missing_reacs(model=cmodel,
                                   missing_genes=gf2_missing_genes,
                                   fasta=tfasta, 
                                   dmnd_db=spdb,
                                   swissprot_map=spmap,
                                   **kwargs)

model = gf2.fill_model(model,mapped_res[0],mapped_res[1],formula_check='existence')
# formula_check,exclude_dna,exclude_rna,idprefix,namespace

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


Running in debugging mode.
fill_model: Running in debugging mode


Adding genes to model: 100%|██████████| 3/3 [00:00<00:00, 5345.33it/s]


In [2]:
gf2._statistics
# possible problem with the current stats: some stuff might be counted twice or more, since the
# current implementation depends on the tables (esp. reactions, since ID depends on NCBI 
# protein accession number and not the uniqueness of the ID)


{'genes': {'missing (before)': 1800,
  'added': 3,
  'missing (after)': 4,
  'no locus tag': 108},
 'reactions': {'added (total)': 7,
  'failed to build': 3,
  'no NCBI, no EC': 0,
  'NCBI, no EC': 43},
 'metabolites': {}}

## the "Filling" part of Gapfilling

**input**

- the model
- missing genes table
- missing reacs table

**out**

- the extended model

**else**

- logging
- save stats information

### Further ideas and Code snippets for the filling part

missing stuff:
- parse BioCyc reaction string
- build biocyc_metabolite
- build biocyc_reaction

### Ideas for collect the stats information

In [7]:
len(cmodel.reactions)

1538